In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# import the table into pandas dataframe
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dataframe  = pd.read_html(url)
dataframe = dataframe[0]

In [3]:
# check the dataframe's size
dataframe.shape

(288, 3)

In [4]:
# quick view to check if we're going well
dataframe.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [5]:
# use groupby function to merge Postcode and Borough columns and then 
# conbine the Neighbourhood values separated by ','
data_gb = dataframe.groupby(['Postcode', 'Borough'], sort=False).agg(lambda x: ','.join(x))
data_gb = data_gb.reset_index()
data_gb.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Not assigned
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Rouge,Malvern"


In [6]:
# replace the 'Queen's Park' Neighborhood
# since set_index in pandas has been desprated, so the best practice here is to use #dataframe.at[]=''
data_gb.at[6, 'Neighbourhood']="Queen's Park"

# delete rows which contained 'Not assigned' 
data = data_gb[data_gb.Borough != 'Not assigned'].reset_index(drop=True)


data.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [7]:
# check the data size
data.shape

(103, 3)

In [8]:
# After trying to use Geocoder, finially decided to use csv file to get the geographical coordinates and
# import the data into dataframe
location = pd.read_csv('Geospatial_Coordinates.csv')
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
# rename the column name and check location datframe's size
location.rename(columns={'Postal Code':'Postcode'}, inplace=True)
location.shape

(103, 3)

In [11]:
# set 'Postcode' as index column so that we can merge these two dataframes easily(both has same column)
location = location.set_index('Postcode').sort_index()

data = data.set_index('Postcode').sort_index()

frames = [data, location]

df = pd.concat(frames, axis=1, join='outer', sort=False)

df.head(10)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476


In [12]:
# reset index of the merged dataframe
df.reset_index(inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
